###### Project

In [165]:
import pandas as pd
#import sqlite3
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import datetime as dt
import folium
from folium import plugins
import geopandas

spray_data = pd.read_csv(
    'Files/spray.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

train_data = pd.read_csv(
    'Files/train.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

weather_data = pd.read_csv(
    'Files/weather.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

test_truncated_data = pd.read_excel(
    'Files/test_truncated.xlsx',
    sheet_name='test_truncated',  # прочитати всі листи
    usecols="A:J"
    #nrows=5
)
DataSets={1:spray_data,
         2:train_data,
         3:weather_data,
         4:test_truncated_data}

#test_truncated_data.head()
#dates=spray_data [['Date'],['Time']]
#train_data.groupby(train_data["Address"])
#dir(spray_data)
#spray_data.amin(axes='Latitude')
#test_truncated_data.info()
#train_data
#weather_data

In [186]:
def add_datetime(train_data):
    train_data['day']=pd.to_datetime(train_data.Date).dt.day
    train_data['weekday']=pd.to_datetime(train_data.Date).dt.weekday
    train_data['month']=pd.to_datetime(train_data.Date).dt.month
    train_data['year']=pd.to_datetime(train_data.Date).dt.year
    return train_data

train_data=add_datetime(train_data)
train_data['RESTUANS']=train_data['Species'].str.contains('RESTUANS')
train_data['PIPIENS']=train_data['Species'].str.contains('PIPIENS')

test_truncated_data=add_datetime(test_truncated_data)
weather_data=add_datetime(weather_data)

In [96]:
geometry = geopandas.points_from_xy(train_data.Longitude, train_data.Latitude)
geo_df = geopandas.GeoDataFrame(
    train_data, geometry=geometry
)

#geo_df.head()

# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]
#geo_df_list

In [97]:
def numMosquitos(draft_year, draft_month):
    numMosquitos=train_data[(geo_df['Trap'] == geo_df.Trap[i]) & (geo_df['month']==draft_month) 
                   & (geo_df['year']==draft_year)]['WnvPresent'].sum()
    return str(numMosquitos)

In [93]:
draft_year_list={2007, 2009, 2013}
draft_month_list={6, 7, 8, 9}

map_ = folium.Map(location=[train_data['Latitude'].mean(),  train_data['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

for draft_year in draft_year_list:
    data_year=train_data[train_data['year']==draft_year].copy()
    #print(data_year.shape)

    for draft_month in draft_month_list:
        #train_data_draft=data_year[(data_year['month']==draft_month)].copy()
        points = (train_data_draft.Latitude.fillna(0),train_data_draft.Longitude.fillna(0))
        train_data_draft=data_year[(data_year['month']==draft_month)].groupby('Trap').sum()
        #points = (train_data_draft.Latitude.fillna(0),train_data_draft.Longitude.fillna(0))
        coordinates =[]
        # Setting lat and long 
        lat = points[0]
        long = points[1]
        #print (train_data_draft.shape)
        #print(long)
    # Append latitude and longitude coordinates array
        train_wnvpresents_Layer=folium.FeatureGroup(name=f'wnvpresents_{draft_year}_{draft_month}')
        train_wnvpresents_Layer.add_to(map_)    
        
        for i, (la, lo) in enumerate(zip(lat,long)):
            #print(la, lo)
            #coordinates.append([la,lo])
            #print (coordinates)
            #print (train_data_draft)
            #wnvpresents=int(train_data_draft[(train_data_draft['Trap'] == train_data_draft.Trap[i])]['WnvPresent'].sum())

            #print(train_data_draft['Trap'])

            #if wnvpresents:
            if geo_df.PIPIENS[i] == True:
                type_color = "green"
            if (geo_df.RESTUANS[i] == True):
                type_color = "red"


            train_wnvpresents_Layer.add_child(
                folium.Marker(
                    location=[la,lo],
                    popup=str(train_data_draft.index[i])+ "<br>"
                    +'wnvpresents '+ str(train_data_draft.WnvPresent[train_data_draft.index[i]])+ "<br>"
                    +'numMosquitos '+ str(train_data_draft.NumMosquitos[train_data_draft.index[i]]),
                    icon=folium.Icon(color="%s" % type_color)
                    )
            )

folium.LayerControl(position='topright', collapsed=False, autoZIndex=False).add_to(map_)
map_.save(f"wnvpresents_.html") 



C:\Users\v.velichko\AppData\Local\Temp\ipykernel_12712\3825397749.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  train_data_draft=data_year[(data_year['month']==draft_month)].groupby('Trap').sum()
C:\Users\v.velichko\AppData\Local\Temp\ipykernel_12712\3825397749.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  train_data_draft=data_year[(data_year['month']==draft_month)].groupby('Trap').sum()
C:\Users\v.velichko\AppData\Local\Temp\ipykernel_12712\3825397749.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to

In [12]:
draft_month=7
draft_year=2007


map_ = folium.Map(location=[train_data['Latitude'].mean(),  train_data['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

shapesLayer=folium.FeatureGroup(name='wnvpresents')
shapesLayer.add_to(map_)

# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:
    if (geo_df.month[i] == draft_month) and  (geo_df.year[i] == draft_year):
        wnvpresents=int(train_data[(geo_df['Trap'] == geo_df.Trap[i]) & (geo_df['month']==draft_month) 
                   & (geo_df['year']==draft_year)]['WnvPresent'].sum())
        if wnvpresents>0:
            if geo_df.PIPIENS[i] == True:
                type_color = "green"
            if (geo_df.RESTUANS[i] == True):
                type_color = "red"
                # Place the markers with the popup labels and data
            shapesLayer.add_child(
                folium.Marker(
                    location=coordinates,
                    popup=geo_df.Trap[i]+ "<br>"
                    +'wnvpresents '+ str(wnvpresents)+ "<br>"+'numMosquitos '+ str(numMosquitos(draft_year, draft_month)),
                    icon=folium.Icon(color="%s" % type_color)
                    )
            )
    i = i + 1
folium.LayerControl(position='topright', collapsed=False, autoZIndex=False).add_to(map_)
map_.save(f"{draft_month}_{draft_year}_map.html") 

In [8]:
train_data_adres = train_data.copy()
train_data_adres.set_index('Address')
train_data_adres.drop(columns=['Date'], axis=1, inplace = True)
train_data_adres.drop(columns=['NumMosquitos'], axis=1, inplace = True)
train_data_adres.drop(columns=['WnvPresent'], axis=1, inplace = True)
train_data_adres.drop(columns=['Species'], axis=1, inplace = True)
#train_data_adres.drop('NumMosquitos', axis=1)
#train_data_adres.drop('AddressAccuracy', axis=1)
#train_data_adres.head()
#res = train_data_adres.apply(lambda x: x.unique().tolist(), axis=1)
#train_data_adres.unique([''])
#train_data_adres.groupby('Address')['Address'].unique()
train_data_adres = train_data_adres[train_data_adres.duplicated()]
train_data_adres.nunique ()
#Dup_Rows.Trap.unique()

Address                   113
Block                      57
Street                    107
Trap                      112
AddressNumberAndStreet    113
Latitude                  113
Longitude                 113
AddressAccuracy             4
RESTUANS                    2
PIPIENS                     2
day                        28
weekday                     5
month                       6
year                        3
geometry                  113
dtype: int64

In [9]:
spray_data['hour']= pd.to_datetime(spray_data.Time,format = '%I:%M:%S %p').dt.hour
spray_data['minute']= pd.to_datetime(spray_data.Time,format = '%I:%M:%S %p').dt.minute

spray_data['day']=pd.to_datetime(spray_data.Date).dt.day
spray_data['weekday']=pd.to_datetime(spray_data.Date).dt.weekday
spray_data['month']=pd.to_datetime(spray_data.Date).dt.month
spray_data['year']=pd.to_datetime(spray_data.Date).dt.year
#spray_data
train_data['day']=pd.to_datetime(train_data.Date).dt.day
train_data['weekday']=pd.to_datetime(train_data.Date).dt.weekday
train_data['month']=pd.to_datetime(train_data.Date).dt.month
train_data['year']=pd.to_datetime(train_data.Date).dt.year
#train_data
weather_data['day']=pd.to_datetime(weather_data.Date).dt.day
#weather_data['weekday']=pd.to_datetime(weather_data.Date).dt.weekday
weather_data['month']=pd.to_datetime(weather_data.Date).dt.month
weather_data['year']=pd.to_datetime(weather_data.Date).dt.year
#weather_data
test_truncated_data['day']=pd.to_datetime(test_truncated_data.Date).dt.day
test_truncated_data['weekday']=pd.to_datetime(test_truncated_data.Date).dt.weekday
test_truncated_data['month']=pd.to_datetime(test_truncated_data.Date).dt.month
test_truncated_data['year']=pd.to_datetime(test_truncated_data.Date).dt.year
#test_truncated_data


In [183]:
def plot_datasetes():
    fig = plt.figure()

    plot_mas = test_truncated_data.groupby(['year'], group_keys=True,
                                           dropna=True).agg({'year': ['count']}).to_dict()[('year', 'count')] 
    plt.bar(plot_mas.keys(), plot_mas.values(), color='blue', label='test_truncated_data counts  in year')
    plt.legend()
    plt.grid(True) 



    fig = plt.figure()
    plot_mas = weather_data.groupby(['year'], group_keys=True,
                                           dropna=True).agg({'year': ['count']}).to_dict()[('year', 'count')] 
    plt.bar(plot_mas.keys(), plot_mas.values(), color='pink', label='weather_data counts  in year')
    plt.legend()
    plt.grid(True)   # лінії допоміжної сітки


    fig = plt.figure()
    #fig2 = plt.figure()

    plot_mas = train_data.groupby(['year'], group_keys=True,
                                           dropna=True).agg({'year': ['count']}).to_dict()[('year', 'count')] 
    plt.bar(plot_mas.keys(), plot_mas.values(), color='green', label='train_data counts  in year')
    plt.legend()
    plt.grid(True)   # лінії допоміжної сітки


    fig = plt.figure()
    plot_mas = spray_data.groupby(['year'], group_keys=True,
                                           dropna=True).agg({'year': ['count']}).to_dict()[('year', 'count')] 
    plt.bar(plot_mas.keys(), plot_mas.values(), color='red', label='spray_data counts  in year')
    plt.legend()
    plt.grid(True)   # лінії допоміжної сітки
    
    fig = plt.figure()
    plot_mas = test_truncated_data.groupby(['year'], group_keys=True,
                                           dropna=True).agg({'year': ['count']}).to_dict()[('year', 'count')] 
    plt.bar(plot_mas.keys(), plot_mas.values(), color='red', label='test_truncated_data counts  in year')
    plt.legend()
    plt.grid(True)   # лінії допоміжної сітки

plot_datasetes()

KeyError: 'year'

<Figure size 640x480 with 0 Axes>

In [ ]:
def bar_chat(_data, grop_list_, agr_colum, agr_func):
    grop_list=[grop_list_]
    agg_func_math = {
        agr_colum: [agr_func]
    }

    plot_mas = _data.groupby(grop_list, group_keys=True, dropna=True).agg(agg_func_math).to_dict()[(agr_colum, agr_func)] 

    fig = plt.figure()
    plt.bar(plot_mas.keys(), plot_mas.values())
    plt.title(f'{grop_list_} {agr_colum} {agr_func} chart')
    
    plt.grid(True)   # лінії допоміжної сітки

In [ ]:
'''grop_list=['Date', 'Address']
bar_chat(train_data, grop_list, 'WnvPresent', 'sum')'''

In [ ]:
#train_data_groupby=train_data.groupby(['Address','Date'])
agg_func_math = {'WnvPresent': ['sum']}

train_data_group=train_data.groupby(['Address','Date']).agg(agg_func_math).copy()
train_data_group

fig = plt.figure()
plot_mas = train_data_group.groupby(['month'], group_keys=True,
                              dropna=True).agg({'WnvPresent': ['sum']}).to_dict()[('WnvPresent', 'sum')] 
plt.bar(plot_mas.keys(), plot_mas.values(), color='red', label='WnvPresent sum month')
plt.legend()

In [1]:
fig = plt.figure()
plot_mas = train_data.groupby(['Date'], group_keys=True,
                              dropna=True).agg({'NumMosquitos': ['sum']}).to_dict()[('NumMosquitos', 'sum')] 
plt.bar(plot_mas.keys(), plot_mas.values(), color='green', label='NumMosquitos')
plt.legend()

plot_mas = train_data.groupby(['Date'], group_keys=True,
                              dropna=True).agg({'WnvPresent': ['sum']}).to_dict()[('WnvPresent', 'sum')] 
plt.bar(plot_mas.keys(), plot_mas.values(), color='red', label='WnvPresent sum Date')
plt.legend()

plt.grid(False)   # лінії допоміжної сітки

NameError: name 'plt' is not defined

In [ ]:
fig = plt.figure()
plot_mas = train_data.groupby(['month'], group_keys=True,
                              dropna=True).agg({'WnvPresent': ['sum']}).to_dict()[('WnvPresent', 'sum')] 
plt.bar(plot_mas.keys(), plot_mas.values(), color='red', label='WnvPresent sum month')
plt.legend()

plt.grid(False)   # лінії допоміжної сітки
#Date
fig = plt.figure()
plot_mas = train_data.groupby(['Date'], group_keys=True,
                              dropna=True).agg({'WnvPresent': ['sum']}).to_dict()[('WnvPresent', 'sum')] 
plt.bar(plot_mas.keys(), plot_mas.values(), color='red', label='WnvPresent sum Date')
plt.legend()

plt.grid(False)   # лінії допоміжної сітки

In [ ]:
train_data_adres.head()

In [ ]:
# Define our key point when map open it
map_ = folium.Map(location=[train_data_adres['Latitude'].mean(),  train_data_adres['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

for la,lo in train_data_adres:
    folium.Marker(
        location=[la,lo],
        icon=folium.Icon(icon_color='white')
    ).add_to(map2)
# Plotting 
map2

In [ ]:
map_ = folium.Map(location=[train_data_adres['Latitude'].mean(),  train_data_adres['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

folium.Marker(train_data_adres['Latitude','Longitude'], popup=train_data_adres['Trap']).add_to(map_)
map_

In [ ]:
#import folium
#Station_1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
#Station_2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

#m = folium.Map(location=[45.5236, -122.6750])
#m = folium.Map(location=[42.391623, -88.089163], tiles="Stamen Terrain", zoom_start=13)


map_ = folium.Map(location=[spray_data['Latitude'].mean(),  spray_data['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

folium.Marker([41.995,  -87.933], popup="CHICAGO O'HARE INTERNATIONAL AIRPORT").add_to(map_)
folium.Marker([41.786,  -88.089163], popup="CHICAGO MIDWAY INTL ARPT").add_to(map_)

folium.CircleMarker(
    location=[spray_data['Latitude'].mean(),  spray_data['Longitude'].mean()],
    radius=10,
    popup="spray_mean",
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(map_)


#train_data_adres
#for index, location_info in train_data.iterrows():
#    folium.Marker([location_info["Latitude"], location_info["Longitude"]], popup=location_info["NumMosquitos"]).add_to(map_)

map_
